## Part I. Preliminary Data Analysis

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
train_1 = pd.read_csv('ACT1_train_450.csv', dtype={"MOLECULE": object, "Act": float})

In [7]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37241 entries, 0 to 37240
Columns: 444 entries, Unnamed: 0 to Act
dtypes: float64(1), int64(443)
memory usage: 126.2 MB


In [8]:
train_1.head(5)

,Unnamed: 0,D_212,D_213,D_214,D_215,D_216,D_217,D_219,D_294,D_295,...,D_7351,D_7352,D_7954,D_7955,D_7957,D_7958,D_8051,D_8053,D_8054,Act
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.0179
1,1,0,0,0,0,0,0,0,4,4,...,0,0,0,0,0,0,0,0,0,4.3003
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.2697
3,3,0,0,0,0,0,0,0,1,0,...,0,0,2,2,3,2,1,1,1,6.1797
4,4,0,0,0,0,0,0,0,0,0,...,2,0,0,2,0,1,0,0,1,4.3003


## Part II. Machine Learning

Split train data into train/test part (80:20)

In [9]:
from sklearn.model_selection import train_test_split
import numpy as np
y = train_1['Act'].values
train_1 = train_1.drop(['Act', 'Unnamed: 0'], axis = 1)
x = train_1.values
X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size = 0.80, random_state = 0)



C:\Users\willi\Anaconda3\envs\data-x\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [11]:
print(np.shape(Y_train), np.shape(X_train), np.shape(Y_test), np.shape(X_test))
Y_train = np.reshape(Y_train,(29792,1))
Y_test = np.reshape(Y_test,(7449,1))

(29792,) (29792, 442) (7449,) (7449, 442)


There are 29792 entries for training the model and 7448 entries for model testing. This split gurantees there's enough data for both the training and testing steps.

### 2. Define evaluation metric r^2

In [12]:
def r_square(X, Y):
    """ r^2 value defined by the competition host, r^2 = 1 indicates 100% prediction accuracy
    """
    avx = np.mean(X)
    avy = np.mean(Y)
    sum1, sumx, sumy = 0, 0, 0
    for i in range(len(X)):
        sum1 += (X[i] - avx)*(Y[i] - avy)
        sumx += (X[i] - avx)*(X[i] - avx)
        sumy += (Y[i] - avy)*(Y[i] - avy)
    print(len(X), sum1, sumx, sumy)
    return sum1*sum1/(sumx*sumy)

### 3. Train neural network

#### 1) Optimized NN with two hidden layers

In [13]:
from tensorflow.keras.models import Sequential
import keras
model = Sequential()

Using TensorFlow backend.


In [14]:
from keras import backend as K
K.clear_session()
#Keras R_2 function
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [15]:
#Keras model
from tensorflow.keras.layers import Dense
from keras.optimizers import adam 

model = keras.models.Sequential()
model.add(keras.layers.core.Dense(units = 50, activation='relu', input_dim = 442))
model.add(keras.layers.core.Dense(25, activation='relu'))
model.add(keras.layers.core.Dense(1, activation = 'relu'))

adam = adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss="mean_squared_error", optimizer=adam, metrics = [coeff_determination])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                22150     
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 23,451
Trainable params: 23,451
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
model.fit(X_train, Y_train, epochs = 500, batch_size= 300)

Epoch 1/500
29792/29792 [==============================] - 0s 17us/step - loss: 2.5169 - coeff_determination: -5.0577
Epoch 2/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.4817 - coeff_determination: -0.1729
Epoch 3/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.3601 - coeff_determination: 0.1239
Epoch 4/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.3050 - coeff_determination: 0.2594
Epoch 5/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.2766 - coeff_determination: 0.3288
Epoch 6/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.2589 - coeff_determination: 0.3700
Epoch 7/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.2452 - coeff_determination: 0.4028
Epoch 8/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.2386 - coeff_determination: 0.4189
Epoch 9/500
29792/29792 [==============================] - 0s 9us/ste

29792/29792 [==============================] - 0s 8us/step - loss: 0.1154 - coeff_determination: 0.7192
Epoch 72/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.1164 - coeff_determination: 0.7168
Epoch 73/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.1224 - coeff_determination: 0.7020
Epoch 74/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.1142 - coeff_determination: 0.7225
Epoch 75/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.1115 - coeff_determination: 0.7280
Epoch 76/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.1126 - coeff_determination: 0.7248
Epoch 77/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.1139 - coeff_determination: 0.7237
Epoch 78/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.1091 - coeff_determination: 0.7346
Epoch 79/500
29792/29792 [==============================] - 0s 9us/step - los

29792/29792 [==============================] - 0s 8us/step - loss: 0.0842 - coeff_determination: 0.7940
Epoch 141/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0878 - coeff_determination: 0.7877
Epoch 142/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0841 - coeff_determination: 0.7955
Epoch 143/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0929 - coeff_determination: 0.7731
Epoch 144/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0831 - coeff_determination: 0.7971
Epoch 145/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0834 - coeff_determination: 0.7964
Epoch 146/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0870 - coeff_determination: 0.7887
Epoch 147/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0832 - coeff_determination: 0.7980
Epoch 148/500
29792/29792 [==============================] - 0s 8us/st

29792/29792 [==============================] - 0s 8us/step - loss: 0.0713 - coeff_determination: 0.8252
Epoch 210/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0693 - coeff_determination: 0.8312
Epoch 211/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0700 - coeff_determination: 0.8303
Epoch 212/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0698 - coeff_determination: 0.8306
Epoch 213/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0703 - coeff_determination: 0.8285
Epoch 214/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0723 - coeff_determination: 0.8244
Epoch 215/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0708 - coeff_determination: 0.8271
Epoch 216/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0703 - coeff_determination: 0.8274
Epoch 217/500
29792/29792 [==============================] - 0s 9us/st

29792/29792 [==============================] - 0s 8us/step - loss: 0.0616 - coeff_determination: 0.8502
Epoch 279/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0655 - coeff_determination: 0.8400
Epoch 280/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0607 - coeff_determination: 0.8517
Epoch 281/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0626 - coeff_determination: 0.8474
Epoch 282/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0622 - coeff_determination: 0.8484
Epoch 283/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0635 - coeff_determination: 0.8453
Epoch 284/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0634 - coeff_determination: 0.8457
Epoch 285/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0635 - coeff_determination: 0.8456
Epoch 286/500
29792/29792 [==============================] - 0s 9us/st

29792/29792 [==============================] - 0s 8us/step - loss: 0.0569 - coeff_determination: 0.8621
Epoch 348/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0573 - coeff_determination: 0.8598
Epoch 349/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0553 - coeff_determination: 0.8653
Epoch 350/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0550 - coeff_determination: 0.8661
Epoch 351/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0591 - coeff_determination: 0.8551
Epoch 352/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0590 - coeff_determination: 0.8561
Epoch 353/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0560 - coeff_determination: 0.8639
Epoch 354/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0577 - coeff_determination: 0.8591
Epoch 355/500
29792/29792 [==============================] - 0s 8us/st

29792/29792 [==============================] - 0s 8us/step - loss: 0.0524 - coeff_determination: 0.8722
Epoch 417/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0530 - coeff_determination: 0.8707
Epoch 418/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0531 - coeff_determination: 0.8707
Epoch 419/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0521 - coeff_determination: 0.8741
Epoch 420/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0526 - coeff_determination: 0.8718
Epoch 421/500
29792/29792 [==============================] - 0s 10us/step - loss: 0.0528 - coeff_determination: 0.8707
Epoch 422/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0517 - coeff_determination: 0.8743
Epoch 423/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0516 - coeff_determination: 0.8747
Epoch 424/500
29792/29792 [==============================] - 0s 10us/

29792/29792 [==============================] - 0s 9us/step - loss: 0.0504 - coeff_determination: 0.8771
Epoch 486/500
29792/29792 [==============================] - 0s 8us/step - loss: 0.0480 - coeff_determination: 0.8832
Epoch 487/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0491 - coeff_determination: 0.8807
Epoch 488/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0491 - coeff_determination: 0.8803
Epoch 489/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0476 - coeff_determination: 0.8844
Epoch 490/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0482 - coeff_determination: 0.8823
Epoch 491/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0481 - coeff_determination: 0.8829
Epoch 492/500
29792/29792 [==============================] - 0s 9us/step - loss: 0.0468 - coeff_determination: 0.8861
Epoch 493/500
29792/29792 [==============================] - 0s 9us/st

In [17]:
print(model.metrics_names)
model.evaluate(X_test, Y_test,verbose=False)

['loss', 'coeff_determination']


[0.21331026846689735, 0.37243172526759644]

In [18]:
import random
import tensorflow as tf


In [19]:
X_placeholder = tf.placeholder(tf.float32, (None, X_train.shape[1])) 
Y_placeholder = tf.placeholder(tf.float32, (None, 1))

features = np.shape(X_train)[1] 
target_size = np.shape(X_train)[0]

learning_rate = 0.001

epochs = 500
batch_size = 300

batch_size_placeholder = tf.placeholder(tf.int64)

# network parameters
n_hidden_1 = 50
n_hidden_2 = 25

ds_train = tf.data.Dataset.from_tensor_slices(( X_placeholder, Y_placeholder )).shuffle(buffer_size=round(len(X_train) * 0.3)).batch(batch_size_placeholder)

ds_test = tf.data.Dataset.from_tensor_slices(( X_placeholder, Y_placeholder )).batch( batch_size_placeholder )

ds_iter = tf.data.Iterator.from_structure(ds_train.output_types, ds_train.output_shapes)

next_x, next_y = ds_iter.get_next()

train_init_op = ds_iter.make_initializer(ds_train)
test_init_op = ds_iter.make_initializer(ds_test)


# initialize weights and bias  
weights = {'w1': tf.Variable(tf.random_normal([features, n_hidden_1])),
           'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
          'out': tf.Variable(tf.random_normal([n_hidden_2, 1]))}

biases = {'b1': tf.Variable(tf.random_normal([n_hidden_1])),
          'b2': tf.Variable(tf.random_normal([n_hidden_2])),
         'out': tf.Variable(tf.random_normal([1]))}

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer 1 with ReLu activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # Hidden layer 2 with ReLu activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # Output layer with ReLu activation
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out_layer
              
# construct model
y_pred = multilayer_perceptron(next_x, weights, biases)

# define cost function(mean squred error) and optimizer(gradient descent)
cost =  tf.losses.mean_squared_error(next_y, y_pred)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize variables
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(epochs):

        sess.run(train_init_op, feed_dict={X_placeholder: X_train, Y_placeholder: Y_train, batch_size_placeholder: batch_size})
        count = 0
        
        while True:
            try:
                count += 1
                _, c = sess.run((optimizer, cost))

            except tf.errors.OutOfRangeError:
                break
        print('Epoch:', (epoch + 1), 'cost =', c)
    
    sess.run(test_init_op, feed_dict={X_placeholder: X_test, Y_placeholder: Y_test, batch_size_placeholder: len(X_test)})

    results, test_cost = sess.run((y_pred, cost))




Epoch: 1 cost = 49692.574
Epoch: 2 cost = 22950.973
Epoch: 3 cost = 13907.3125
Epoch: 4 cost = 9994.961
Epoch: 5 cost = 7774.3438
Epoch: 6 cost = 4456.8022
Epoch: 7 cost = 4298.754
Epoch: 8 cost = 2994.0618
Epoch: 9 cost = 2822.3394
Epoch: 10 cost = 2283.803
Epoch: 11 cost = 2346.4282
Epoch: 12 cost = 1309.527
Epoch: 13 cost = 1516.4093
Epoch: 14 cost = 1371.1549
Epoch: 15 cost = 906.09937
Epoch: 16 cost = 726.1133
Epoch: 17 cost = 799.061
Epoch: 18 cost = 466.204
Epoch: 19 cost = 603.21765
Epoch: 20 cost = 419.94904
Epoch: 21 cost = 304.7427
Epoch: 22 cost = 214.50662
Epoch: 23 cost = 289.50766
Epoch: 24 cost = 169.44313
Epoch: 25 cost = 137.05338
Epoch: 26 cost = 134.96996
Epoch: 27 cost = 117.62736
Epoch: 28 cost = 73.88362
Epoch: 29 cost = 80.62346
Epoch: 30 cost = 25.95968
Epoch: 31 cost = 41.528843
Epoch: 32 cost = 16.94266
Epoch: 33 cost = 51.15832
Epoch: 34 cost = 41.777668
Epoch: 35 cost = 81.86895
Epoch: 36 cost = 18.278353
Epoch: 37 cost = 17.298576
Epoch: 38 cost = 12.71744

Epoch: 294 cost = 0.29494712
Epoch: 295 cost = 0.19010113
Epoch: 296 cost = 0.24471669
Epoch: 297 cost = 0.6278726
Epoch: 298 cost = 0.24016069
Epoch: 299 cost = 0.17640388
Epoch: 300 cost = 0.19429417
Epoch: 301 cost = 0.20456791
Epoch: 302 cost = 0.18189086
Epoch: 303 cost = 0.2573383
Epoch: 304 cost = 0.24544258
Epoch: 305 cost = 0.14416611
Epoch: 306 cost = 0.24755234
Epoch: 307 cost = 0.5415604
Epoch: 308 cost = 0.15560602
Epoch: 309 cost = 0.21013658
Epoch: 310 cost = 0.58359885
Epoch: 311 cost = 0.18604071
Epoch: 312 cost = 0.23514594
Epoch: 313 cost = 0.37765262
Epoch: 314 cost = 0.34434932
Epoch: 315 cost = 0.25825283
Epoch: 316 cost = 0.20018885
Epoch: 317 cost = 0.16801217
Epoch: 318 cost = 0.3795107
Epoch: 319 cost = 0.19656706
Epoch: 320 cost = 0.20695198
Epoch: 321 cost = 0.23246041
Epoch: 322 cost = 0.34097606
Epoch: 323 cost = 0.2993862
Epoch: 324 cost = 0.20562622
Epoch: 325 cost = 0.23193015
Epoch: 326 cost = 0.2325999
Epoch: 327 cost = 0.27356485
Epoch: 328 cost = 0.

In [20]:
#    print(results)
    print(test_cost)
    print('R^2:', r_square(np.reshape(results, (len(results),)),Y_test))

0.35232526
7449 [1894.64937025] 3014.8221829004497 [3185.02459734]
R^2: [0.3738378]
